1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
 - 토큰 수 초과로 답변을 생성하지 못할 수 있고
 - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

# 1. 문서의 내용을 읽는다

In [ ]:
%pip install python-docx


In [ ]:
from docx import Document

document = Document("./tax.docx")
print(f'document == {dir(document)}')
full_text = ''
for index, paragraph in enumerate(document.paragraphs):
    print(f'paragraph == {paragraph.text}')
    full_text += f'{paragraph.text}\n'

# 2.문서를 쪼갠다

In [ ]:
%pip install tiktoken

In [ ]:
import tiktoken

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)
    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)

    return text_list

In [ ]:
chunk_list = split_text(full_text, 1500)

# 3.문서 임베딩

In [ ]:
%pip install chromadb

In [ ]:
import chromadb
chroma_client = chromadb.Client()


In [ ]:
collection_name = 'tax_collection'
tax_collection = chroma_client.create_collection(collection_name)

In [ ]:
import os
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(api_key=openai_api_key, model_name='text-embedding-3-large')

In [ ]:
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)

In [ ]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')

len(id_list)

In [ ]:
len(chunk_list)

In [ ]:
tax_collection.add(documents=chunk_list, ids=id_list)

# 4.유사도 검색

In [31]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_doc = tax_collection.query(query_texts=query, n_results=3)

In [32]:
retrieved_doc['documents'][0]

['가. 「국가유공자 등 예우 및 지원에 관한 법률」 또는 「보훈보상대상자 지원에 관한 법률」에 따라 받는 보훈급여금ㆍ학습보조비 및 「북한이탈주민의 보호 및 정착지원에 관한 법률」에 따라 받는 정착금ㆍ보로금(報勞金)과 그 밖의 금품\n나. 「국가보안법」에 따라 받는 상금과 보로금\n다. 「상훈법」에 따른 훈장과 관련하여 받는 부상(副賞)이나 그 밖에 대통령령으로 정하는 상금과 부상\n라. 종업원등 또는 대학의 교직원이 퇴직한 후에 사용자등 또는 산학협력단으로부터 지급받거나 대학의 학생이 소속 대학에 설치된 산학협력단으로부터 받는 직무발명보상금으로서 대통령령으로 정하는 금액. 다만, 직무발명보상금을 지급한 사용자등 또는 산학협력단과 대통령령으로 정하는 특수관계에 있는 자가 받는 직무발명보상금은 제외한다.\n마. 「국군포로의 송환 및 대우 등에 관한 법률」에 따라 국군포로가 받는 위로지원금과 그 밖의 금품\n바. 「문화유산의 보존 및 활용에 관한 법률」에 따라 국가지정문화유산으로 지정된 서화ㆍ골동품의 양도로 발생하는 소득\n사. 서화ㆍ골동품을 박물관 또는 미술관에 양도함으로써 발생하는 소득\n아. 제21조제1항제26호에 따른 종교인소득 중 다음의 어느 하나에 해당하는 소득\n\u3000\u3000\u3000\u30001) 「통계법」 제22조에 따라 통계청장이 고시하는 한국표준직업분류에 따른 종교관련종사자(이하 “종교관련종사자”라 한다)가 받는 대통령령으로 정하는 학자금\n\u3000\u3000\u3000\u30002) 종교관련종사자가 받는 대통령령으로 정하는 식사 또는 식사대\n\u3000\u3000\u3000\u30003) 종교관련종사자가 받는 대통령령으로 정하는 실비변상적 성질의 지급액\n\u3000\u3000\u3000\u30004) 종교관련종사자 또는 그 배우자의 출산이나 6세 이하(해당 과세기간 개시일을 기준으로 판단한다) 자녀의 보육과 관련하여 종교단체로부터 받는 금액으로서 월 20만원 이내의 금액\n\u3000\u3000\u3000\u30005) 종교

# 5. LLM 질의

In [33]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요."},
    {"role": "user", "content": query}
  ]
)

In [36]:
response.choices[0].message.content

'연봉 5천만원인 직장인의 소득세를 계산하려면, 먼저 과세표준을 구하고 다음으로 적용 세율을 계산해야 합니다. 한국의 소득세는 누진세 구조로, 과세표준 구간에 따라 차등적으로 세율이 적용됩니다.\n\n1. **총 급여에서 비과세 항목을 제외한 근로소득금액을 구합니다:**\n   - 만약 특별한 비과세 항목이 없다고 가정하면, 총 급여 5,000만원에서 비과세 항목을 0원으로 가정하겠습니다.\n\n2. **필요경비 공제(근로소득공제)를 적용합니다:**\n   - 근로소득공제는 연봉 구간에 따라 일정 비율이 적용됩니다.\n   - 5,000만원 기준으로는 먼저 1,500만원 구간까지 70%, 그 다음 1,500만원 초과 ~ 4,500만원 이하 구간까지 40%, 그 다음 4,500만원 초과 ~ 10,000만원 이하 구간까지는 15%가 적용됩니다.\n   \n   ```\n   1,500만원까지: 1,500만원 × 70% = 1,050만원\n   1,500만원 초과 4,500만원 이하: (4,500만원 - 1,500만원) × 40% = 1,200만원\n   4,500만원 초과: (5,000만원 - 4,500만원) × 15% = 75만원\n   ```\n   \n   총 근로소득공제액 = 1,050만원 + 1,200만원 + 75만원 = 2,325만원\n\n3. **총 급여에서 근로소득공제를 차감한 근로소득금액을 구합니다:**\n   ```\n   총 급여 5,000만원 - 근로소득공제 2,325만원 = 근로소득금액 2,675만원\n   ```\n\n4. **근로소득금액에서 종합소득공제를 적용하여 과세표준을 구합니다:**\n   - 기본 인적공제 (본인 1,500만원)과 각종 추가 인적공제 및 기타 공제 항목을 적용합니다.\n   - 가정으로 기본 인적공제만 적용한다고 하면:\n\n   ```\n   근로소득금액 2,675만원 - 기본 인적공제 1,500만원 = 과세표준 1,175만원\n   ```\n\n5. **과세표준에 대한 세율을 적용하여 소득세를 계산합니다:**\